## Parameter Setup

In [7]:
import subprocess
import numpy as np
import os; os.chdir('../src') # For server
from datetime import datetime

# Parameters for all methods
method = 'FedProx'
mu_value = 0.5
local_epochs_list = [25, 50, 75, 100, 125, 150]  # Different number of local epochs to test
global_lr_fedprox = 1/mu_value + 1/9.13
local_lr_fedprox = 1/(9.13+mu_value)
random_seed = 1
R_combined = [1.0]

num_clients = 10
num_samples = 200
input_dim = 10
num_classes = 2
data_dir = '../data/fedprox_syndata/test'
output_data_dir = '../results/test'
stopping_threshold = -1
num_rounds = 20  # Number of rounds

# Parameters for data generation
num_devices = num_clients
x_dim = input_dim
b_dim = num_classes

## Run the Model and Collect Results

In [8]:
start_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(f"Start time: {start_time}")

# Loop through each choice of R
for R in R_combined:
    # Generate data for the current R with specified dimensions and number of devices
    print(f"Generating data for R={R} with {num_devices} devices...")
    subprocess.run(f"python simulation/generate_data.py --R {R} --num_devices {num_devices} --n_samples {num_samples} "
                   f"--x_dim {x_dim} --b_dim {b_dim} --seed {random_seed} "
                   f"--output_dir {data_dir}", shell=True, check=True)

    # Loop through each method
    for local_epochs in local_epochs_list:
        print(f"Running {method} with mu={mu_value} and local_epochs={local_epochs}...")
        
        # Generate a unique output directory or file name based on the local_epochs
        specific_output_dir = f"{output_data_dir}/fedprox_mu{mu_value}_epochs{local_epochs}_R{R}"
        
        # Create the output directory if needed
        subprocess.run(f"mkdir -p {specific_output_dir}", shell=True)
    
        # Run the simulation with the specified local epochs
        subprocess.run(
            f"python simulation/simulation_main.py --method {method} --num_clients {num_clients} "
            f"--lr_global {global_lr_fedprox} --lr_local {local_lr_fedprox} --mu {mu_value} "
            f"--input_dim {input_dim} --num_classes {num_classes} --local_epochs {local_epochs} "
            f"--data_dir {data_dir} --output_data_dir {specific_output_dir} "
            f"--stopping_threshold {stopping_threshold} --num_rounds {num_rounds} "
            f"--record_error stat", shell=True, check=True)
            

end_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(f"End time: {end_time}")

## Visualize the Results

In [9]:
# FOR Total Error Visualizaiton
# Convert the lists into space-separated strings
R_value = R_combined[0]
local_epochs_str = ' '.join(map(str, local_epochs_list))

# Command for subprocess
command = f"python simulation/visualize_local_epochs.py --output_data_dir {output_data_dir} --R_value {R_value} --mu {mu_value} --local_epochs_list {local_epochs_str}"

# Run the command using subprocess
subprocess.run(command, shell=True, check=True)